# Autoencoder Development

#### 1. Dependencies

This section could include:
* Library imports.
* Constant definitions

In [1]:
# library imports
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import time
from IPython.display import clear_output
import cv2

In [2]:
# detect if CUDA is available, if not use CPU
gpu_available = torch.cuda.is_available()

if gpu_available:
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

# Declare global constants
PAD_TOP = 5
PAD_BOTTOM = 5
PAD_LEFT = 3
PAD_RIGHT = 4
P_HEIGHT = 150 + PAD_TOP + PAD_BOTTOM
P_WIDTH = 225 + PAD_LEFT + PAD_RIGHT

FIRST = 16
SECOND = 32
THIRD = 64

#### 2. Data Loading & Feature Engineering

This section could include:

* Loading of data files
* Data manipulation
* Feature engineering strategies

In [3]:
# load images
images1 = np.load("subset_1.npy")
images2 = np.load("subset_2.npy")
images3 = np.load("subset_3.npy")

images = np.array(list(images1) + list(images2) + list(images3))

# add padding
def pad(images):
    padded_images = []
    for i in range(len(images)):
        img = np.reshape(images[i], (150, 225, 3))
        padded = cv2.copyMakeBorder(img, PAD_TOP, PAD_BOTTOM, PAD_LEFT, PAD_RIGHT, borderType = cv2.BORDER_CONSTANT, value = [0, 0, 0])
        padded_images.append(padded)

    images = np.array(padded_images)
    return images

images = pad(images)

# split into training and testing datasets & prepare for use by neural networks
num_images = images.shape[0]
training_samples = int((0.75 * num_images) // 1)

images = images.reshape(-1, 3, P_HEIGHT, P_WIDTH)
np.random.shuffle(images)

train_data = images[:training_samples] / 255.
test_data = images[training_samples:] / 255.

train_data = torch.tensor(train_data, dtype = torch.float32)
test_data = torch.tensor(test_data, dtype = torch.float32)

#### 3. Model Development

This section could include:

* Model Definitions
* Hyperparameter Tuning
* Training code
* Optimisation

In [4]:
# Handles creation of encoder and decoder
class Autoencoder(nn.Module):
    def __init__(self, encoder, *network_parameters):
        super(Autoencoder, self).__init__()
        self.encoder = encoder
        if network_parameters:
            self.network_parameters = network_parameters[0]
        else:
            self.network_parameters = None

        # If network parameters are provided, use them to create the model
        if self.encoder:
            self.encode = self.create_model()
            if self.network_parameters and len(self.network_parameters) == 2:
                try:
                    self.encode.load_state_dict(torch.load(self.network_parameters[0], map_location = DEVICE))
                    self.encode.to(DEVICE)
                    self.trained = True
                    print("Parameters", self.network_parameters[0], "loaded successfully")
                except Exception as err:
                    print("Error loading model")
                    print(err)
                    print("Creating new model")
                    self.encode.to(DEVICE)
                    self.trained = False
            else:
                self.encode.to(DEVICE)
                self.trained = False
                
        else:
            self.decode = self.create_model()
            if self.network_parameters and len(self.network_parameters) == 2:
                try:
                    self.decode.load_state_dict(torch.load(self.network_parameters[1], map_location = DEVICE))
                    self.decode.to(DEVICE)
                    self.trained = True
                    print("Parameters", self.network_parameters[1], "loaded successfully")
                except Exception as err:
                    print("Error loading model")
                    print(err)
                    print("Creating new model")
                    self.decode.to(DEVICE)
                    self.trained = False
            else:
                self.decode.to(DEVICE)
                self.trained = False

    
    def create_model(self):
        if self.encoder:
            model = nn.Sequential(
                nn.Conv2d(in_channels = 3, out_channels = FIRST, kernel_size = 2, stride = 2, padding = 0),
                nn.LeakyReLU(0.01),

                nn.Conv2d(in_channels = FIRST, out_channels = SECOND, kernel_size = 2, stride = 2, padding = 0),
                nn.LeakyReLU(0.01),

                nn.Conv2d(in_channels = SECOND, out_channels = THIRD, kernel_size = 2, stride = 2, padding = 0),
            )
        else:
            model = nn.Sequential(
                nn.ConvTranspose2d(in_channels = THIRD, out_channels = SECOND, kernel_size = 2, stride = 2, padding = 0),
                nn.LeakyReLU(0.01),

                nn.ConvTranspose2d(in_channels = SECOND, out_channels = FIRST, kernel_size = 2, stride = 2, padding = 0),
                nn.LeakyReLU(0.01),

                nn.ConvTranspose2d(in_channels = FIRST, out_channels = 3, kernel_size = 2, stride = 2, padding = 0),
                nn.Sigmoid()
            )
        
        return model


    # encode and decode methods, used by neural networks
    def enc(self, x):
        return self.encode(x)
    
    def dec(self, x):
        return self.decode(x)
    

    # encode and decode methods, used for application by user
    # takes an image as a numpy array (101250, ) and returns compressed data in a numpy array
    def encode_data(self, image):
        if self.trained:
            if not self.encoder:
                print("Decoder cannot encode image")
                return None
            else:
                try:
                    original_size = image.nbytes
                    print("Original image size:", original_size / 1e6, "MB")
                    image = np.reshape(image, (150, 225, 3)) / 255.
                    image_data = cv2.copyMakeBorder(image, PAD_TOP, PAD_BOTTOM, PAD_LEFT, PAD_RIGHT, borderType = cv2.BORDER_CONSTANT, value = [0, 0, 0])
                    image_data = image_data.reshape(3, P_HEIGHT, P_WIDTH)
                    data = torch.tensor(image_data, dtype = torch.float32)
                    data = data.to(DEVICE)
                    output = self.enc(data)
                    output = output.cpu().detach().numpy().astype(np.float16)
                    print("Compressed image size:", output.nbytes / 1e6, "MB")
                    print("Compression ratio:", round(original_size / output.nbytes, 5))

                    return output
                except Exception as err:
                    print("Image is not in the correct format")
                    print(err)
                    return None
        else:
            print("Encoder has not been trained yet")
            return None

    # takes compressed data in a numpy array and returns it as an image in a numpy array (3, 152, 228)
    def decode_data(self, data):
        if self.trained:
            if self.encoder:
                print("Decoder cannot decode data")
                return None
            else:
                try:
                    data = torch.tensor(data, dtype = torch.float32)
                    data = data.to(DEVICE)
                    output = self.dec(data)
                    output = output.cpu().detach().numpy()
                    output_image = (output * 255).astype(np.uint8)
                    output_image = np.reshape(output_image, (P_HEIGHT, P_WIDTH, 3))
                    clipped_image = output_image[PAD_TOP:PAD_TOP + 150, PAD_LEFT:PAD_LEFT + 225]

                    return clipped_image
                except Exception as err:
                    print("Data is not in the correct format")
                    print(err)
                    return None
        else:
            print("Decoder has not been trained yet")
            return None


#### 4. Model Evaluation

This section could include:

* Testing of models trained
* Generation of results

In [5]:
# Handels autoencoder training
class AutoencoderTrainer:
    def __init__(self, encoder, decoder, learning_rate, loss_f):
        self.encoder = encoder
        self.decoder = decoder
        self.learning_rate = learning_rate
        self.loss_f = loss_f
        self.optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr = self.learning_rate)

    def autoencode(self, encoder, decoder, data):
        return decoder.dec(encoder.enc(data))

    # calculate loss using selected loss function
    def calculate_loss(self, output, target):
        loss = self.loss_f(output, target)
        return loss
    
    # reset, calculate and update gradients
    def update(self, loss):
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    # train method
    def train(self, epochs):
        if self.encoder.trained or self.decoder.trained:
            print("Encoder and/or decoder have already been trained")
            print("Create a new encoder and decoder instance to train")
            return None
        else:
            self.losses = []
            for epoch in range(1, epochs + 1):
                for sample in train_data:
                    sample = sample.to(DEVICE)
                    output = self.autoencode(self.encoder, self.decoder, sample)  
                    
                    loss = self.calculate_loss(output, sample)
                    self.update(loss)

                print(f"Epoch: {epoch}, Loss: {loss.item()}")
                self.losses.append(loss.item())

            self.encoder.trained = True
            self.decoder.trained = True

    def test(self):
        if self.encoder.trained and self.decoder.trained:
            outputs = []
            for sample in test_data:
                sample = sample.to(DEVICE)
                output = self.autoencode(self.encoder, self.decoder, sample)
                outputs.append(output)

            return outputs
        else:
            print("Encoder and/or decoder have not been trained yet")
            return None

In [ ]:
# Create fresh encoder and decoder instances
# encoder = Autoencoder(True)
# decoder = Autoencoder(False)

# uncomment to load pre-trained model
network_parameters = ["encoder_parameters.pth", "decoder_parameters.pth"]
encoder = Autoencoder(True, network_parameters)
decoder = Autoencoder(False, network_parameters)

# Train autoencoder and decoder
lr = 0.001 # 0.0001
epochs = 10 # 600
loss_f = nn.MSELoss()
trainer = AutoencoderTrainer(encoder, decoder, lr, loss_f)
trainer.train(epochs)

In [ ]:
# neural network structure
print(encoder)
print(decoder)

#### 5. Figure Creation

This section could include:

* Creation of figures for the report.
* Creationg of tables for the report.

In [ ]:
# manual application from user compress and unpack image
images1 = np.load("subset_1.npy")
images2 = np.load("subset_2.npy")
images3 = np.load("subset_3.npy")

images = np.array(list(images1) + list(images2) + list(images3))


i = np.random.randint(0, len(images))
img = images[i]

# encode_data takes an image as a numpy array (101250, ) and returns compressed data in a numpy array
# decode_data takes compressed data in as a numpy array and returns it
# as a numpy array (160, 232, 3) ready for plt.show()
encoded = encoder.encode_data(img)
decoded = decoder.decode_data(encoded)
plt.axis("off")
plt.imshow(decoded)

In [ ]:
# process entire test_dataset through autoencoder
imgs = trainer.test()

for i in range(len(imgs)):
    imgs[i] = (imgs[i].detach().cpu().numpy() * 255).astype(np.uint8)
    imgs[i] = np.reshape(imgs[i].flatten(), (P_HEIGHT, P_WIDTH, 3))
    imgs[i] = imgs[i][PAD_TOP:PAD_TOP + 150, PAD_LEFT:PAD_LEFT + 225]

print("number of test images:", len(imgs))

In [ ]:
# display a random original image followed by the autoencoded version (1 second delay) for 5 images in test_data
image_indexes = []
for i in range(5):
    num = np.random.randint(0, len(test_data) - 1)
    image_indexes.append(num)
    clear_output(wait = True)
    test_image = np.reshape((test_data[num].numpy() * 255).astype(np.uint8), (P_HEIGHT, P_WIDTH, 3))
    original_test_image = test_image[PAD_TOP:PAD_TOP + 150, PAD_LEFT:PAD_LEFT + 225]
    plt.imshow(original_test_image)
    plt.axis("off")
    plt.show()
    time.sleep(1)
    clear_output(wait = True)
    plt.imshow(imgs[num])
    plt.axis("off")
    plt.show()
    time.sleep(1)
print(image_indexes)

In [28]:
# Compare a specific image vs autoencoded image from the test dataset
IMAGE_NUMBER = 20

In [ ]:
plt.axis("off")
compare_original = np.reshape((test_data[IMAGE_NUMBER].numpy() * 255).astype(np.uint8), (P_HEIGHT, P_WIDTH, 3))
compare_original = compare_original[PAD_TOP:PAD_TOP + 150, PAD_LEFT:PAD_LEFT + 225]
plt.imshow(compare_original)

In [ ]:
plt.axis("off")
compare_autoencoded = imgs[IMAGE_NUMBER]
compare_autoencoded = compare_autoencoded[PAD_TOP:PAD_TOP + 150, PAD_LEFT:PAD_LEFT + 225]
plt.imshow(compare_autoencoded)

In [ ]:
# compare performance of bce vs mse
bce_encoder = Autoencoder(True)
bce_decoder = Autoencoder(False)

mse_encoder = Autoencoder(True)
mse_decoder = Autoencoder(False)

lr = 0.001
bce_trainer = AutoencoderTrainer(bce_encoder, bce_decoder, lr, nn.BCELoss())
mse_trainer = AutoencoderTrainer(mse_encoder, mse_decoder, lr, nn.MSELoss())

epochs = 15 # 90
bce_trainer.train(epochs)
mse_trainer.train(epochs)

In [ ]:
# compare bce vs mse
bce_outputs = bce_trainer.test()
mse_outputs = mse_trainer.test()

for b in range(len(bce_outputs)):
    bce_outputs[b] = (bce_outputs[b].detach().cpu().numpy() * 255).astype(np.uint8)
    bce_outputs[b] = np.reshape(bce_outputs[b].flatten(), (P_HEIGHT, P_WIDTH, 3))

for m in range(len(mse_outputs)):
    mse_outputs[m] = (mse_outputs[m].detach().cpu().numpy() * 255).astype(np.uint8)
    mse_outputs[m] = np.reshape(mse_outputs[m].flatten(), (P_HEIGHT, P_WIDTH, 3))

In [ ]:
num = 50

In [ ]:
print("Original image")
plt.axis("off")
plt.imshow(test_data[num].cpu().detach().numpy().reshape(P_HEIGHT, P_WIDTH, 3)[PAD_TOP:PAD_TOP + 150, PAD_LEFT:PAD_LEFT + 225])

In [ ]:
print("BCE Loss")
plt.axis("off")
plt.imshow(bce_outputs[num][PAD_TOP:PAD_TOP + 150, PAD_LEFT:PAD_LEFT + 225])

In [ ]:
print("MSELoss")
plt.axis("off")
plt.imshow(mse_outputs[num][PAD_TOP:PAD_TOP + 150, PAD_LEFT:PAD_LEFT + 225])

In [ ]:
# save model parameters
def save_parameters(encoder_parameter_file_name, decoder_parameter_file_name):
    encoder_parameters = torch.save(encoder.encode.state_dict(), encoder_parameter_file_name)
    decoder_parameters = torch.save(decoder.decode.state_dict(), decoder_parameter_file_name)

# save_parameters("new_encoder_parameters.pth", "new_decoder_parameters.pth")

In [ ]:
# 1: lr = 0.0001, epochs = 600
# Autoencoder(
#   (encode): Sequential(
#     (0): Conv2d(3, 16, kernel_size=(2, 2), stride=(2, 2))
#     (1): LeakyReLU(negative_slope=0.01)
#     (2): Conv2d(16, 32, kernel_size=(2, 2), stride=(2, 2))
#     (3): LeakyReLU(negative_slope=0.01)
#     (4): Conv2d(32, 64, kernel_size=(2, 2), stride=(2, 2))
#   )
# )
# Autoencoder(
#   (decode): Sequential(
#     (0): ConvTranspose2d(64, 32, kernel_size=(2, 2), stride=(2, 2))
#     (1): LeakyReLU(negative_slope=0.01)
#     (2): ConvTranspose2d(32, 16, kernel_size=(2, 2), stride=(2, 2))
#     (3): LeakyReLU(negative_slope=0.01)
#     (4): ConvTranspose2d(16, 3, kernel_size=(2, 2), stride=(2, 2))
#     (5): Sigmoid()
#   )
# )

# encoder_parameters = torch.save(encoder.encode.state_dict(), "encoder_parameters.pth")
# decoder_parameters = torch.save(decoder.decode.state_dict(), "decoder_parameters.pth")

In [ ]:
# 2
# Autoencoder(
#   (encode): Sequential(
#     (0): Conv2d(3, 8, kernel_size=(2, 2), stride=(2, 2))
#     (1): LeakyReLU(negative_slope=0.01)
#     (2): Conv2d(8, 16, kernel_size=(2, 2), stride=(2, 2))
#   )
# )
# Autoencoder(
#   (decode): Sequential(
#     (0): ConvTranspose2d(16, 8, kernel_size=(2, 2), stride=(2, 2))
#     (1): LeakyReLU(negative_slope=0.01)
#     (2): ConvTranspose2d(8, 3, kernel_size=(2, 2), stride=(2, 2))
#     (3): Sigmoid()
#   )
# )

# encoder_parameters = torch.save(encoder.encode.state_dict(), "encoder_parameters_overfitted.pth")
# decoder_parameters = torch.save(decoder.decode.state_dict(), "decoder_parameters_overfitted.pth")

In [ ]:
# 3: lr = 0.0001, epochs = 800
# Autoencoder(
#   (encode): Sequential(
#     (0): Conv2d(3, 8, kernel_size=(2, 2), stride=(2, 2))
#     (1): LeakyReLU(negative_slope=0.01)
#     (2): Conv2d(8, 24, kernel_size=(2, 2), stride=(2, 2))
#     (3): LeakyReLU(negative_slope=0.01)
#     (4): Conv2d(24, 48, kernel_size=(2, 2), stride=(2, 2))
#   )
# )
# Autoencoder(
#   (decode): Sequential(
#     (0): ConvTranspose2d(48, 24, kernel_size=(2, 2), stride=(2, 2))
#     (1): LeakyReLU(negative_slope=0.01)
#     (2): ConvTranspose2d(24, 8, kernel_size=(2, 2), stride=(2, 2))
#     (3): LeakyReLU(negative_slope=0.01)
#     (4): ConvTranspose2d(8, 3, kernel_size=(2, 2), stride=(2, 2))
#     (5): Sigmoid()
#   )
# )

# encoder_parameters = torch.save(encoder.encode.state_dict(), "encoder_parameters_overfitted2.pth")
# decoder_parameters = torch.save(decoder.decode.state_dict(), "decoder_parameters_overfitted2.pth")